In [13]:
import nltk

In [14]:
example='vsd is good boy and i love to work with him'

In [15]:
tokens=nltk.word_tokenize(example)
print(tokens)

['vsd', 'is', 'good', 'boy', 'and', 'i', 'love', 'to', 'work', 'with', 'him']


In [16]:
tagged=nltk.pos_tag(tokens)
print(tagged)

[('vsd', 'NN'), ('is', 'VBZ'), ('good', 'JJ'), ('boy', 'NN'), ('and', 'CC'), ('i', 'JJ'), ('love', 'NN'), ('to', 'TO'), ('work', 'VB'), ('with', 'IN'), ('him', 'PRP')]


In [17]:
nltk.download('maxent_ne_chunker')
nltk.download('words')
entities=nltk.chunk.ne_chunk(tagged)
entities.pprint()

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\udayg\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\udayg\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


(S
  vsd/NN
  is/VBZ
  good/JJ
  boy/NN
  and/CC
  i/JJ
  love/NN
  to/TO
  work/VB
  with/IN
  him/PRP)


In [18]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [19]:
sia=SentimentIntensityAnalyzer()

In [20]:
sia.polarity_scores(example)

{'neg': 0.0, 'neu': 0.53, 'pos': 0.47, 'compound': 0.7964}

In [21]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

In [22]:
from scipy.special import softmax

In [23]:
MODEL="cardiffnlp/twitter-roberta-base-sentiment"

In [24]:
tokenizer=AutoTokenizer.from_pretrained(MODEL)
model=AutoModelForSequenceClassification.from_pretrained(MODEL)

In [25]:
example= '''I like it most redmi note 12 pro +5g",
      "It's very amazing",
      "Overall good phone. But it has its usual miui bugs. Could've given lpddr5 Ram and ufs 3.1 storage and a snapdragon processor.. Would've demolished competitor brands then at this price range. Redmi note series always tries to imitate iPhone design.. But even though this phone tries to look like the latest iPhone, it's still an eye catching design. Stereo speakers are one of the best I've seen in smartphones.. Dolby Atmos is an added bonus that's it.. Pwm dimming really helps.. There's no more eye straining even after long hours of viewing.. Don't expect much from camera.. It's not upto the mark for the price they're charging. Not the phone for heavy gamers.. But it gives a very good multimedia experience with the speakers and display.. It's an apt phone for a normal user wanting to upgrade.. But for above average user, save up some money and buy Samsung, OnePlus or any other company.",
      "After three months of continuous use, this mobile phone and its camera are not satisfactory.\nCompany claims to have a 200 megapixel camera but after trying it in various ways it doesn't seem to get that kind of a result and the screen rotation feature and some other features don't seem to be working properly.\ud83d\ude4f\ud83c\udffc\ud83d\ude13",
      "Battery drainage and hitting issue",
      "Front camera is not goood. Avoid for front camera else is ok.",
      "The product is really good but battery backup could be better.",
      "It overheats while playing the game BGMI(PUBG), it says it has a cooling chamber, but it's not working....\nRemaining all fine'''

In [26]:
def result(example):
    text=tokenizer(example,return_tensors='pt')
    output=model(**text)
    scores=output[0][0].detach().numpy()
    score=softmax(scores)
    return score


In [27]:
example_2=  '''
  "I order this phone a day ago and found a dead spot on the screen. After calling them; they suggested to visit Samsung service center as it has clearly mentioned that the product has 7 Days of service center replacement warranty(A Big Scam). I reached to the service centre and waited for around more than 2 hrs for my turn to come and finally they simply denied to give any kind of service for refurbished products from the Amazon. Finally my suggestion is not to fall in these kind of fake assurance. I have raised a complaint on consumer forum for the same.",
  "The Samsung service is poor, the terms and conditions regarding warranty are not adhered to. The company attributes the difective screen issue on mishandling on the part of customer, thereby charging huge bill for such repairs, that too in warranty period.
"
'''

In [28]:
print(result(example_2))

[0.9330508  0.06157485 0.00537428]


In [29]:
# import numpy
# reviews_1=score[2]- score[0]
# ratings_1 =4.2/5
# price_1=27000
# reviews_2=score3[2]-score3[0]
# ratings_2=2/5
# price_2=94000
# # reviews_1=reviews_1.astype(numpy.int64)
# # reviews_2=reviews_2.astype(numpy.int64)

# normalised_price_1=(price_1-min(price_1,price_2))/(max(price_1,price_2)-min(price_1,price_2))
# normalised_price_2=(price_2-min(price_1,price_2))/(max(price_1,price_2)-min(price_1,price_2))
# result1= 0.4*ratings_1 - 0.2*normalised_price_1 + 0.4*reviews_1
# result2= 0.4*ratings_2 - 0.2*normalised_price_2 + 0.4*reviews_2
# print(result1)
# print(result2)

In [30]:
import json
with open('product_details.json', 'r') as file:
    data = json.load(file)

In [31]:
def remove_surrogates(text):
    return text.encode('utf-16', 'surrogatepass').decode('utf-16')
rating_list = []
title_list = []
price_list = []
polarity_scores=[]
for item in data:
    if 'reviews' in item:
        reviews = item['reviews']
        for review in reviews:
            clean_text = remove_surrogates(review)
        polarity_scores.append(result(clean_text))
    if 'price' in item:
        price = item['price']
        price_list.append(price)

    if 'rating' in item:
        rating = item['rating']
        rating_list.append(rating)  
    if 'title' in item:
        title = item['title']
        title_list.append(title)    


            
print(polarity_scores,price_list,rating_list,title_list)

[] ['1,39,999', '16,999', '6,999', '8,499', '7,699'] ['4.4 out of 5 stars', '3.7 out of 5 stars', '4.0 out of 5 stars', '4.1 out of 5 stars', '4.1 out of 5 stars'] ['Samsung Galaxy S24 Ultra 5G AI Smartphone (Titanium Black, 12GB, 512GB Storage)', 'Redmi Note 13 5G (Arctic White, 6GB RAM, 128GB Storage) | 5G Ready | 120Hz Bezel-Less AMOLED | 7.mm Slimmest Note Ever | 108MP Pro-Grade Camera', 'POCO C65 (Matte Black 4GB RAM 128GB Storage)', 'Redmi 13C (Stardust Black, 6GB RAM, 128GB Storage) | Powered by 4G MediaTek Helio G85 | 90Hz Display | 50MP AI Triple Camera', 'Redmi 13C (Starshine Green, 4GB RAM, 128GB Storage) | Powered by 4G MediaTek Helio G85 | 90Hz Display | 50MP AI Triple Camera']


In [32]:
price_list_int = []
for price in price_list:
    price=int(price.replace(',', ''))
    price_list_int.append(price)
print(price_list_int)
min_price = min(price_list_int)
max_price = max(price_list_int)
final_dict = {}
for i in range(len(title_list)):
        rating = float(rating_list[i].split()[0])

        result = (0.4 * rating -
                  0.2 * (price_list_int[i] - min_price) / (max_price - min_price) +
                  0.4 * (polarity_scores[i][2] - polarity_scores[i][0]))
        
        final_dict[title_list[i]] = result


highest_key = None
highest_value = -100 

for key, value in final_dict.items():
    if value > highest_value:
        highest_value = value
        highest_key = key


print(f"The product with the highest score is: {highest_key} with a score of {highest_value}")



[139999, 16999, 6999, 8499, 7699]


IndexError: list index out of range